In [2]:
def find_cycles_efficient(dependency_data):
    """Identifies cyclic dependencies using constrained depth-first search."""
    
    def build_dependency_graph(modules):
        """Constructs adjacency list from module dependencies."""
        return {
            module: set(module_data.get("imports", [])) & modules.keys()
            for module, module_data in modules.items()
        }

    def detect_cycles_from_node(graph, start_node, max_cycles=10, max_depth=20):
        """Finds cycles originating from a specific node with safety limits."""
        cycle_results = []
        processing_stack = [(start_node, [start_node])]
        
        while processing_stack and len(cycle_results) < max_cycles:
            current_node, path = processing_stack.pop()
            
            for neighbor in list(graph.get(current_node, []))[:max_cycles*2]:  # Limit neighbor checks
                if neighbor in path:
                    cycle_start = path.index(neighbor)
                    detected_cycle = path[cycle_start:] + [neighbor]
                    if not any(c == detected_cycle for c in cycle_results):
                        cycle_results.append(detected_cycle)
                        if len(cycle_results) >= max_cycles:
                            break
                elif len(path) < max_depth:
                    processing_stack.append((neighbor, path + [neighbor]))
        
        return cycle_results

    def get_limited_starting_nodes(graph, max_nodes=50):
        """Selects initial nodes for cycle detection with quantity limit."""
        return list(graph.keys())[:max_nodes]

    # Main execution flow
    dependency_graph = build_dependency_graph(dependency_data)
    candidate_nodes = get_limited_starting_nodes(dependency_graph)
    
    unique_cycles = []
    for node in candidate_nodes:
        if len(unique_cycles) >= 20:  # Total cycle limit
            break
        for cycle in detect_cycles_from_node(dependency_graph, node):
            if cycle not in unique_cycles:
                unique_cycles.append(cycle)
                if len(unique_cycles) >= 20:
                    break
    
    return unique_cycles


In [3]:
def analyze_dependency_depth_efficient(dependency_data):
    """Analyzes dependency depth for a sample of top-level modules."""
    
    def build_dependency_graph(modules):
        """Constructs adjacency list from dependency data."""
        return {
            module: set(module_data.get("imports", [])) & modules.keys()
            for module, module_data in modules.items()
        }

    def calculate_max_depth(graph, start_module, max_depth_limit=10):
        """Calculates maximum dependency depth using BFS with depth bounding."""
        depth_tracker = {start_module: 0}
        processing_queue = [(start_module, 0)]
        
        while processing_queue:
            current_module, current_depth = processing_queue.pop(0)
            
            if current_depth >= max_depth_limit:
                continue
                
            for dependency in graph.get(current_module, []):
                if dependency not in depth_tracker or depth_tracker[dependency] < current_depth + 1:
                    depth_tracker[dependency] = current_depth + 1
                    processing_queue.append((dependency, current_depth + 1))
        
        return max(depth_tracker.values(), default=0)

    def get_top_level_modules(module_names, sample_size=20):
        """Identifies top-level modules without submodule notation."""
        return [name for name in module_names if '.' not in name][:sample_size]

    # Main execution flow
    dependency_graph = build_dependency_graph(dependency_data)
    sampled_modules = get_top_level_modules(dependency_graph.keys())
    
    return {
        module: calculate_max_depth(dependency_graph, module)
        for module in sampled_modules
    }

In [4]:
def identify_module_relationships(dependency_data):
    """Identifies unused and isolated modules from dependency data."""
    
    def build_module_sets(modules):
        """Constructs tracking sets for module relationships."""
        referenced_modules = set()  # Modules imported by others
        modules_with_dependencies = set()  # Modules that import others
        
        for module, data in modules.items():
            imports = data.get("imports", [])
            if imports:
                modules_with_dependencies.add(module)
                
            # Track modules that are imported by others
            for imported_module in imports:
                if imported_module in modules:
                    referenced_modules.add(imported_module)
                    
        return referenced_modules, modules_with_dependencies

    # Get all analyzed modules
    all_modules = set(dependency_data.keys())
    
    # Build relationship sets
    imported_modules, dependent_modules = build_module_sets(dependency_data)
    
    # Calculate unused modules (defined but never imported)
    unused = all_modules.difference(imported_modules)
    
    # Calculate disconnected modules (no imports and not imported)
    disconnected = all_modules.difference(
        dependent_modules.union(imported_modules)
    )

    # Return results with limited sample size
    return {
        "unused_modules": sorted(list(unused))[:30],  # Sorted for consistency
        "disconnected_modules": sorted(list(disconnected))[:30]
    }


In [7]:
import json

def load_dependency_data(file_path='dependencies.json'):
    """Loads dependency data from JSON file with safety limits."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            
            return data
    except FileNotFoundError:
        raise SystemExit("Error: dependencies.json file not found")

def execute_analysis_workflow(data):
    """Orchestrates the analysis pipeline and result formatting"""
    results = {}
    
    print("\nIdentifying cyclic dependencies...")
    results['cycles'] = find_cycles_efficient(data)
    
    print("\nAnalyzing dependency depths...")
    results['depths'] = analyze_dependency_depth_efficient(data)
    
    print("\nDetecting unused components...")
    results['unused'] = identify_module_relationships(data)
    
    return results

def display_analysis_results(results):
    """Formats and displays analysis outcomes"""
    print("\n" + "="*40 + " RESULTS " + "="*40)
    
    # Cycle dependencies
    print(f"\nFound {len(results['cycles'])} cyclic dependencies (showing 5):")
    for i, cycle in enumerate(results['cycles'][:5], 1):
        print(f"{i}. {' -> '.join(cycle)}")
    
    # Depth analysis
    print("\nDeepest dependency chains (top 5):")
    for module, depth in sorted(results['depths'].items(), 
                                key=lambda x: x[1], reverse=True)[:5]:
        print(f"- {module}: {depth} levels deep")
    
    # Unused/Disconnected modules
    print(f"\nUnused modules ({len(results['unused']['unused_modules'])} total):")
    print("Sample:", ", ".join(results['unused']['unused_modules'][:5]))
    
    print(f"\nDisconnected modules ({len(results['unused']['disconnected_modules'])} total):")
    print("Sample:", ", ".join(results['unused']['disconnected_modules'][:5]))

if __name__ == "__main__":
    try:
        dependency_data = load_dependency_data()
        analysis_results = execute_analysis_workflow(dependency_data)
        display_analysis_results(analysis_results)
        
    except MemoryError:
        print("Memory limit exceeded. Consider:")
        print("- Using a data subset")
        print("- Increasing system memory")
        print("- Optimizing analysis parameters")
    except Exception as e:
        print(f"Unexpected error: {str(e)}")



Identifying cyclic dependencies...

Analyzing dependency depths...

Detecting unused components...

======================================== RESULTS ========================================

Found 20 cyclic dependencies (showing 5):
1. requests -> requests
2. urllib3 -> urllib3
3. requests -> requests.sessions -> requests.models -> requests.utils -> requests
4. requests -> requests.sessions -> requests.utils -> requests
5. requests -> requests.utils -> requests

Deepest dependency chains (top 5):
- __main__: 10 levels deep
- cryptography: 10 levels deep
- requests: 10 levels deep
- urllib3: 10 levels deep
- charset_normalizer: 0 levels deep

Unused modules (1 total):
Sample: __main__

Disconnected modules (0 total):
Sample: 


In [10]:
cycles = analysis_results['cycles']
depth_analysis = analysis_results['depths']

def display_dependency_cycles(cycles):
    """Formats and displays cyclic dependencies with visual structure"""
    if not cycles:
        print("\nNo cyclic dependencies detected in the module graph")
        return

    print(f"\nFound {len(cycles)} dependency cycles:")
    print("=" * 80)
    
    for idx, cycle in enumerate(cycles, 1):
        cycle_str = " -> ".join(cycle)
        print(f"Cycle {idx}:")
        print(f"  {cycle_str}")
        print("-" * 60)

def display_dependency_depths(depth_data):
    """Presents dependency depth analysis in ranked format"""
    if not depth_data:
        print("\nNo depth data available")
        return

    print("\nDependency Chain Depth Ranking:")
    print("=" * 80)
    
    ranked = sorted(depth_data.items(), key=lambda x: (-x[1], x[0]))
    
    for rank, (module, depth) in enumerate(ranked[:10], 1):
        print(f"{rank}. {module:<40} (Depth: {depth})")
    if len(ranked) > 10:
        print(f"\nShowing top 10 of {len(ranked)} analyzed modules")

# Usage example:
print("\n" + "="*80)
print(" DEPENDENCY ANALYSIS RESULTS ".center(80, "="))
print("="*80)

display_dependency_cycles(cycles)
display_dependency_depths(depth_analysis)



========================= DEPENDENCY ANALYSIS RESULTS ==========================

Found 20 dependency cycles:
Cycle 1:
  requests -> requests
------------------------------------------------------------
Cycle 2:
  urllib3 -> urllib3
------------------------------------------------------------
Cycle 3:
  requests -> requests.sessions -> requests.models -> requests.utils -> requests
------------------------------------------------------------
Cycle 4:
  requests -> requests.sessions -> requests.utils -> requests
------------------------------------------------------------
Cycle 5:
  requests -> requests.utils -> requests
------------------------------------------------------------
Cycle 6:
  requests -> requests.models -> requests.utils -> requests
------------------------------------------------------------
Cycle 7:
  requests -> requests.api -> requests
------------------------------------------------------------
Cycle 8:
  requests -> requests.api -> requests.sessions -> requests.mod